In [2]:
from google.colab import drive

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
!pip install pandas

In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [6]:
df= pd.read_csv('/content/drive/MyDrive/dataset.csv', sep="\t")

df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,timestamp,IP Address
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,82850235,BlackVue DR600GW-PMP,Mobile_Electronics,5,0,0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1440987567,193.93.167.87
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,82850235,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0,1,N,Y,five star,it's great,2015-08-31,1441001998,193.93.167.87
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0,0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,1440959483,193.93.167.87
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,82850235,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4,0,0,N,Y,Work very well,Work very well,2015-08-31,1441015251,193.93.167.87
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0,0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31,1440973308,205.10.168.66
5,US,7846966,RY8DDL22YG4R5,B00KA6CCVY,563475445,Aweek® Air Acoustic Earpiece Headset for Baofe...,Mobile_Electronics,3,0,1,N,Y,"It appears to be good, but I'm still waiting f...","It appears to be good, but I'm still waiting f...",2015-08-31,1440977588,205.10.168.66
6,US,21299354,R2AT2426ZHFUHH,B00MJCDPM2,563475445,Sentey LS-4460 B-Trek S8 Bluetooth Portable St...,Mobile_Electronics,3,0,1,N,Y,Didn't love the first one,"First one arrived as a brick. Wouldn't work, ...",2015-08-31,1441033322,205.10.168.66
7,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,563475445,BlackVue DR600GW-PMP,Mobile_Electronics,5,0,0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1441012613,205.10.168.66
8,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0,0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1440987567,206.33.134.174


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,timestamp,IP Address
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,82850235,BlackVue DR600GW-PMP,Mobile_Electronics,5,0,0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,1440987567,193.93.167.87
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,82850235,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0,1,N,Y,five star,it's great,2015-08-31,1441001998,193.93.167.87
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0,0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,1440959483,193.93.167.87
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,82850235,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4,0,0,N,Y,Work very well,Work very well,2015-08-31,1441015251,193.93.167.87
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0,0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31,1440973308,205.10.168.66


In [9]:
df.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date', 'timestamp',
       'IP Address'],
      dtype='object')

In [10]:
df['marketplace'].value_counts()

marketplace
US    9
Name: count, dtype: int64

In [11]:
df['label'] = list(range(len(df.index)))

In [12]:
model = LogisticRegression(class_weight='balanced')

In [13]:
print(df['label'].value_counts())

label
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
Name: count, dtype: int64


In [14]:
print(df['verified_purchase'].value_counts())

verified_purchase
Y    9
Name: count, dtype: int64


In [15]:
df['label'] = np.where(df['verified_purchase'] == 'Y', 1, 0)

# Check if there are samples from both classes
if len(df['label'].unique()) < 2:
    # Add some fake reviews to the dataset
    df_fake = df.copy()
    df_fake['verified_purchase'] = 'N'
    df_fake['review_body'] = df_fake['review_body'] + ' This is a fake review.'
    df = pd.concat([df, df_fake])

    # Recalculate the 'label' column
    df['label'] = np.where(df['verified_purchase'] == 'Y', 1, 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review_body'], df['label'], test_size=0.2, random_state=42)

In [16]:

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'\W|\d', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize
    words = nltk.word_tokenize(text)
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Join the words
    text = ' '.join(words)
    return text

df['review_body'] = df['review_body'].apply(preprocess_text)

# Create a new column 'label' to indicate whether the review is genuine or fake
df['label'] = np.where(df['verified_purchase'] == 'Y', 1, 0)

# Ensure that we have samples from both classes in the 'label' column
if len(df['label'].unique()) < 2:
    raise ValueError("The 'label' column does not contain samples from both classes. Please verify your data and preprocessing steps.")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review_body'], df['label'], test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train machine learning models
models = [
    ('NB', MultinomialNB()),
    ('LR', LogisticRegression()),
    ('SVM', SVC())
]

for name, model in models:
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    print(f'Model: {name}')
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(classification_report(y_test, y_pred))
    print()


Model: NB
Accuracy: 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       4.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0


Model: LR
Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4


Model: SVM
Accuracy: 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       4.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [17]:
product_mapping = df[['product_title', 'product_category']].drop_duplicates()
product_mapping['product_category'] = product_mapping.groupby('product_category').cumcount()
product_mapping = product_mapping.set_index('product_title')['product_category'].to_dict()

In [18]:
def text_process(review, product_title):
    nopunc = [char for char in review if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [product_title] + [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [19]:
!pip install scikit-learn
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pandas as pd

# Define a function for text processing
def text_process(text):
    # Your text processing logic here
    return text

# Define the pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel='linear'))
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['product_title'], df['label'], test_size=0.2, random_state=42)

# Train the SVM model using the pipeline
pipeline.fit(X_train, y_train)

# Predict on the test data
filtered_reviews = pipeline.predict(X_test)

# Print the filtered reviews for each product
for i, product in enumerate(filtered_reviews):
    print("Predicted label for review " + str(i+1) + ": " + str(product))
    print("Review: " + X_test.iloc[i])  # Print the review text
    print("\n")


Predicted label for review 1: 0
Review: BlackVue DR600GW-PMP


Predicted label for review 2: 0
Review: GENSSI GSM / GPS Two Way Smart Phone Car Alarm System for iPhone Android Security


Predicted label for review 3: 0
Review: BlackVue DR600GW-PMP


Predicted label for review 4: 0
Review: Aweek® Air Acoustic Earpiece Headset for Baofeng Two Way Radios UV-5R UV-B6 BF-888S UV-B6 UV-B5 Walkie Talkies 2-Pin Jack




In [21]:
# prompt: write to print reviews

for i, product in enumerate(filtered_reviews):
    print("Predicted label for review " + str(i+1) + ": " + str(product))
    print("Review: " + X_test.iloc[i])  # Print the review text
    print("\n")


Predicted label for review 1: 0
Review: BlackVue DR600GW-PMP


Predicted label for review 2: 0
Review: GENSSI GSM / GPS Two Way Smart Phone Car Alarm System for iPhone Android Security


Predicted label for review 3: 0
Review: BlackVue DR600GW-PMP


Predicted label for review 4: 0
Review: Aweek® Air Acoustic Earpiece Headset for Baofeng Two Way Radios UV-5R UV-B6 BF-888S UV-B6 UV-B5 Walkie Talkies 2-Pin Jack




In [22]:
# prompt: print the review content separetly from the dataset

for i, review in enumerate(df['review_body']):
    print("Review " + str(i+1) + ": " + review)
    print("\n")


Review 1: advertised everything work perfectly happy camera matter fact going buy another one nd car


Review 2: great


Review 3: work great fit life proof case iphone


Review 4: work well


Review 5: careful product bought several camera image pretty decent battery hold charge


Review 6: appears good still waiting item radio battery charger go star


Review 7: first one arrived brick work charge useless exchanged new one work advertised


Review 8: advertised everything work perfectly happy camera matter fact going buy another one nd car


Review 9: advertised everything work perfectly happy camera matter fact going buy another one nd car


Review 10: advertised everything work perfectly happy camera matter fact going buy another one nd car fake review


Review 11: great fake review


Review 12: work great fit life proof case iphone fake review


Review 13: work well fake review


Review 14: careful product bought several camera image pretty decent battery hold charge fake review



In [23]:
# prompt: print separetly for every product

for product, reviews in df.groupby('product_title'):
    print(f"Reviews for product '{product}':")
    for review in reviews['review_body']:
        print(f"\t- {review}")
    print("\n")


Reviews for product 'Aweek® Air Acoustic Earpiece Headset for Baofeng Two Way Radios UV-5R UV-B6 BF-888S UV-B6 UV-B5 Walkie Talkies 2-Pin Jack':
	- appears good still waiting item radio battery charger go star
	- appears good still waiting item radio battery charger go star fake review


Reviews for product 'BlackVue DR600GW-PMP':
	- advertised everything work perfectly happy camera matter fact going buy another one nd car
	- advertised everything work perfectly happy camera matter fact going buy another one nd car
	- advertised everything work perfectly happy camera matter fact going buy another one nd car
	- advertised everything work perfectly happy camera matter fact going buy another one nd car fake review
	- advertised everything work perfectly happy camera matter fact going buy another one nd car fake review
	- advertised everything work perfectly happy camera matter fact going buy another one nd car fake review


Reviews for product 'GENSSI GSM / GPS Two Way Smart Phone Car Ala

In [24]:
# prompt: like the above the product should contain only genuine review

# Filter the data to only include genuine reviews
df_genuine = df[df['label'] == 1]

# Group the genuine reviews by product title
product_reviews = df_genuine.groupby('product_title')['review_body'].apply(list)

# Print the genuine reviews for each product
for product, reviews in product_reviews.items():
    print(f"Genuine reviews for product '{product}':")
    for review in reviews:
        print(f"\t- {review}")
    print("\n")


Genuine reviews for product 'Aweek® Air Acoustic Earpiece Headset for Baofeng Two Way Radios UV-5R UV-B6 BF-888S UV-B6 UV-B5 Walkie Talkies 2-Pin Jack':
	- appears good still waiting item radio battery charger go star


Genuine reviews for product 'BlackVue DR600GW-PMP':
	- advertised everything work perfectly happy camera matter fact going buy another one nd car
	- advertised everything work perfectly happy camera matter fact going buy another one nd car
	- advertised everything work perfectly happy camera matter fact going buy another one nd car


Genuine reviews for product 'GENSSI GSM / GPS Two Way Smart Phone Car Alarm System for iPhone Android Security':
	- great


Genuine reviews for product 'Generic Car Dashboard Video Camera Vehicle Video Accident Recorder (2.0" 1080P)':
	- careful product bought several camera image pretty decent battery hold charge


Genuine reviews for product 'Sentey LS-4460 B-Trek S8 Bluetooth Portable Stereo Speaker (Black)':
	- first one arrived brick w